# Installazione libreria deap

In [ ]:
pip install deap

# Import librerie

In [ ]:
import array
import random
import math

import numpy
import time

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

# Apertura File

In [ ]:
#lettura del file
File = open("c.txt","r")
Lines = File.readlines()

<h1>Lettura della istanza</h1>
<p> Il primo rigo contiene le informazioni relative al numero di libri B, numero di librerie L e numero di giorni D.
La riga successiva contiene lo score associato ad ogni libro, utilizziamo un array chiamato Books_score così definito:
Books_score -> chiave = libro, valore = score.

Successivamente ci sono solo righe dedicate alle librerie, la prima riga contiene come informazioni il numero di libri nella libreria, la durata del periodo di signup e il numero di libri scannerizzati ogni giorno.
<br>Il rigo successivo contiene tutti gli ID dei libri in possesso a quella liberia. In questo caso abbiamo utilizzato un array multidimensionale così definito:

<ol> <li>Library -> chiave = lib_id, valore = (num_of_books, signup_days, num_of_books_per_day, books) dove:
        <ul> <li>num_of_books - numero di libri presenti nella libreria,</li>
             <li>signup_days  - numero di giorni necessari per il signup della libreria, </li>
             <li>num_of_books_per_day - numero massimo di libri che possono essere fotocopiati in un giorno nella libreria,</li>
             <li>books - lista di libri presenti nella libreria. </li>
        </ul> </li>
</ol>

In [ ]:
Split_Line = Lines[0].split(" ")

Num_Of_Books = int(Split_Line[0])
Num_Of_Libraries = int(Split_Line[1])
Num_Of_Days = int(Split_Line[2])

Books_score = {}
Split_Line = Lines[1].split(" ")
for i in range(Num_Of_Books) :
    Books_score[i] = int(Split_Line[i])

Library = {}

Line_number = 2
for j in range(Num_Of_Libraries) :
    Split_Line = Lines[Line_number].split(" ")

    Library[j] = (int(Split_Line[0]), int(Split_Line[1]), int(Split_Line[2]), [] )

    Split_Line = Lines[Line_number + 1].split(" ")
    for p in Split_Line:
        Library[j][3].append(int(p))

    #ordino i libri per score crescente
    Library[j][3].sort(key = lambda x: Books_score[x], reverse=True)
    Line_number = Line_number + 2

size = Num_Of_Libraries

# Imostazioni Dep




*   creator.create("FitnessMax", base.Fitness, weights=(1.0,)): creazione classe FitnessMax indicando una fitness a massimizzare
*   creator.create("Individual", list, fitness=creator.FitnessMax): crazione classe Individual con attributo FitnessMax
*   toolbox.register("indices", random.sample, range(IND_SIZE), IND_SIZE): crea un alias chiamato "indices" all'interno della toolbox. L'alias "indices" è associato alla funzione random.sample, che viene utilizzata per generare una sequenza casuale di IND_SIZE elementi presi da un range che va da 0 a IND_SIZE-1. Quindi, chiamando toolbox.indices(), si otterrà una nuova sequenza casuale ogni volta.
*   toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices): crea un alias chiamato "individual" nella toolbox. L'alias "individual" è associato alla funzione tools.initIterate. Questa funzione viene utilizzata per inizializzare un oggetto di tipo creator.Individual utilizzando la sequenza generata dalla funzione toolbox.indices. Quindi, chiamando toolbox.individual(), si otterrà un nuovo individuo inizializzato con una sequenza casuale.
*   toolbox.register("population", tools.initRepeat, list, toolbox.individual): crea un alias chiamato "population" nella toolbox. L'alias "population" è associato alla funzione tools.initRepeat. Questa funzione viene utilizzata per inizializzare una popolazione di individui. La popolazione è rappresentata come una lista di individui. La funzione tools.initRepeat viene chiamata con gli argomenti list (per indicare che la popolazione sarà rappresentata come una lista) e toolbox.individual (per indicare che la funzione toolbox.individual sarà utilizzata per inizializzare ogni individuo nella popolazione).

Richiamando la toolbox.populatio(n) riusciremo a creare una popolazione di n individui inizializzati con sequenze casuali.









In [ ]:
#imposto la fitness con modalità FitnessMax
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
#creazione classe individual con attributo fitnessmax
creator.create("Individual", list, fitness=creator.FitnessMax)

IND_SIZE = size


toolbox = base.Toolbox()

#creo alias indices, sequenza generata dalla random_sample
toolbox.register("indices", random.sample, range(IND_SIZE), IND_SIZE)
#creo alias individual
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
#creo alias population
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Rappresentazione soluzione
La soluzione consiste nel ordine in cui veniene eseguito il signup delle varie librerie, quindi la soluzione è rappresentata da un vettore di numeri unici che vanno da 0 a size-1, cioè il numero di librerie lette nel file.
Funzione obbiettivo
Prende in input uno stato, l'array che rappresenta una soluzione del problema, per ogni libreria:

Tramite la variabile Current day si tiene conto del periodo di sign up e se la libreria può iniziare a lavorare.
Si calcola quanti libri possono essere scannerizzati fino alla fine dei D giorni
Un ciclo For itera tutti i libri della lista, tenendo conto di quelli già scannerizzati e del numero massimo di libri che si possono scannerizzare.

In [ ]:
#definizione della funzione evalOneMax per il problema dell'hascode
def evalOneMax(individual):
    Current_day = 0
    Number_of_Scanned_Books = 0
    Total_Score = 0
    Total_Copied_Book = 0
    Scanned_Book = {} # cliave libro, valore libreria

    for lib_id in individual:
        Current_day += Library[lib_id][1] #viene aggiunto il signup days
        if Current_day >= Num_Of_Days:
            break

        #ordino i libri per score crescente
        Library[lib_id][3].sort()
        Max_Num_Book = (Num_Of_Days - Current_day - 1) * Library[lib_id][2] #numero di libri scannerizzati ogni giorno
        Copied_Book  = 0

        for book_id in Library[lib_id][3]:
            #se non ho ancora scannerizzato il libro ed esiste
            if (book_id not in Scanned_Book) and (book_id in Books_score):
                #salvo il libro scannerizzato
                Scanned_Book[book_id] = lib_id
                #aggiorno punteggio
                Total_Score += Books_score[book_id]

                Copied_Book       += 1
                Total_Copied_Book += 1
                if Copied_Book > Max_Num_Book:
                    break
    return Total_Score,

# Operatori di deap



*   toolbox.register("evaluate", evalOneMax):  Questa riga crea un alias chiamato "evaluate" nella toolbox. L'alias "evaluate" è associato alla funzione evalOneMax. Questa funzione è utilizzata per valutare la fitness di un individuo.
*   toolbox.register("mate", tools.cxTwoPoint): Questa riga crea un alias chiamato "mate" nella toolbox. L'alias "mate" è associato alla funzione tools.cxTwoPoint. Questa funzione esegue un crossover a due punti sugli individui dati in input. Chiamando toolbox.mate(individual1, individual2), si otterranno due nuovi individui generati attraverso il crossover.
*   toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.02): Questa riga crea un alias chiamato "mutate" nella toolbox. L'alias "mutate" è associato alla funzione tools.mutShuffleIndexes. Questa funzione esegue una mutazione sugli attributi dell'individuo dato in input. La mutazione utilizza il metodo di "mescolare" gli attributi dell'individuo per introdurre variazioni casuali. L'argomento indpb=0.02 indica la probabilità di mutazione per ogni attributo dell'individuo. Chiamando toolbox.mutate(individual), si otterrà un nuovo individuo con alcuni attributi mutati.
*   toolbox.register("select", tools.selTournament, tournsize = int(IND_SIZE/2)): Questa riga crea un alias chiamato "select" nella toolbox. L'alias "select" è associato alla funzione tools.selTournament. Questa funzione esegue la selezione degli individui utilizzando il metodo del "torneo". Nel torneo, vengono selezionati casualmente diversi individui dalla popolazione e viene scelto il migliore tra di loro come risultato della selezione. L'argomento tournsize specifica la dimensione del torneo, ovvero il numero di individui coinvolti in ogni torneo. In questo caso, tournsize è impostato a int(IND_SIZE/2), quindi il torneo coinvolgerà la metà degli individui nella popolazione. Chiamando toolbox.select(population, k), si otterrà una nuova popolazione di individui selezionati dal torneo. L'argomento k specifica il numero di individui da selezionare.



In [ ]:
#indico quale funzione usare per la valutazione
toolbox.register("evaluate", evalOneMax)
#uso cxTwoPoint per il crossover. (Esegue un crossover a due punti sugli individui della sequenza di input)
toolbox.register("mate", tools.cxTwoPoint)
# uso di mutShuffleIndexes per la mutazione. (Mescola gli attributi dell'individuo di input e restituisce il mutante)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.02)
#uso di selDoubleTournament per la selezione degli individui
toolbox.register("select", tools.selTournament, tournsize =int(IND_SIZE/2))

# Esecuzione algoritmo
## uso dell' algoritmo eaSimple :
Inizializzazione: L'algoritmo inizia generando una popolazione di individui casuali.La popolazione iniziale viene creata utilizzando la funzione di inizializzazione registrata nella toolbox.

Valutazione: Per ogni individuo della popolazione, viene calcolata la sua fitness utilizzando la funzione di valutazione registrata nella toolbox.

Riproduzione: L'algoritmo seleziona gli individui più adatti dalla popolazione per la riproduzione. La selezione viene effettuata utilizzando la funzione di selezione registrata nella toolbox. Gli individui selezionati vengono accoppiati utilizzando l'operatore di crossover registrato nella toolbox, che combina le caratteristiche degli individui genitori per creare nuovi individui.

Mutazione: Alcuni degli individui generati attraverso il crossover vengono sottoposti a una mutazione casuale. La mutazione introduce variazioni casuali negli individui al fine di esplorare nuove regioni dello spazio delle soluzioni. La mutazione viene eseguita utilizzando l'operatore di mutazione registrato nella toolbox.

Valutazione dei figli: Dopo il crossover e la mutazione, i nuovi individui ("figli") vengono valutati per determinare la loro fitness.

Sopravvivenza: L'algoritmo seleziona gli individui sopravvissuti per la prossima generazione. Questa selezione può essere basata sulla fitness dei genitori e dei figli. Gli individui con una fitness migliore hanno una maggiore probabilità di essere selezionati per la sopravvivenza. L'operatore di selezione registrato nella toolbox viene utilizzato per eseguire questa selezione.

Ripeti: I passaggi vengono ripetuti per un certo numero di generazioni.

Output: Alla fine dell'algoritmo, viene restituito il risultato migliore ottenuto durante l'esecuzione.

  

In [ ]:
#metriche da mostrare
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

#dimensione e generazione della popolazione
POP_SIZE = 100
pop = toolbox.population(n = POP_SIZE)

tic = time.time()

#uso dell'algoritmo eaSimple
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.3, ngen=4000,
                                   stats=stats, halloffame=hof, verbose=True)

tac = time.time()
tempo = float(tac-tic) / 60

print("--- time: "+str(tempo)+" minuti---")

<table><thead><tr><th>Istanza</th><th>Risultato Tabu</th><th>Tempo di calcolo </th><th>Risultato Genetico</th><th>Tempo di calcolo</th><th>Risultato concorrenti 530th</th><th>Tempo di calcolo</th></tr></thead><tbody><tr><td>A - Example</td><td>21</td><td>0.1 sec</td><td>21</td><td>0.02 sec</td><td>21</td><td>0.003 sec</td></tr><tr><td rowspan="3">B - Read On</td><td>20 iter - 5.822.900</td><td>5 sec </td><td rowspan="3">5.893.800<br><br>popsize = 150<br>ngen   = 200<br>cxpb   = 0.3<br>mutpb = 0.7<br></td><td rowspan="3">1 min per arrivare<br>intorno a <br>5.800.000, <br>convergenza a<br>circa 10 min</td><td rowspan="3">5.822.900</td><td rowspan="15">Tempo <br>che varia dai <br>15 ai 20 minuti <br>per ogni <br>dataset.</td></tr><tr><td>100 iter - 5.822.900</td><td>18 sec </td></tr><tr><td>200 iter - 5.822.900</td><td>35 sec </td></tr><tr><td rowspan="3">C - incunabula</td><td>20 iter - 5.648.064</td><td>7 min </td><td rowspan="3">1.737.140<br>popsize = 100<br>ngen    = 200<br>cxpb    = 0.5<br>mutpb  = 0.7<br></td><td rowspan="3">in un 1 min <br>intorno al <br>1.300.000, <br>convergenza in <br>circa 10 min</td><td rowspan="3">5.690.378</td></tr><tr><td>100 iter - 5.652.179</td><td>40 min</td></tr><tr><td>200 iter - 5.652.830</td><td>76 min</td></tr><tr><td rowspan="3">D - Tough choices</td><td>20 iter - 4.823.260</td><td>38 min </td><td rowspan="3">4.415.000<br>pop_size = 100<br>ngen = 50<br>cxpb = 0.5<br>mutpb = 0.7<br></td><td rowspan="3">1 min per <br>arrivare intorno a<br>4.300.000, <br>converge in <br>circa 10 min</td><td rowspan="3">5.028.530</td></tr><tr><td>60 iter - 4.831.775</td><td>2 h </td></tr><tr><td>None <br></td><td></td></tr><tr><td rowspan="3">E - So many books</td><td>20 iter - 5.031.624</td><td>3 min </td><td rowspan="3">3.275.190<br>pop_size = 100<br>ngen = 400<br>cxpb = 0.5<br>mutpb = 0.7<br></td><td rowspan="3">circa 2 min per <br>arrivare a <br>2.500.000, <br>converge in <br>circa 10 min  </td><td rowspan="3">5.034.898</td></tr><tr><td>100 iter - 5.064.292</td><td>14 min </td></tr><tr><td>200 iter - 5.083.813</td><td>28 min </td></tr><tr><td rowspan="3">F - Libraries of the <br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;world </td><td>20 iter - 5.267.148</td><td>1.5 min </td><td rowspan="3">5.259.020<br>pop_size = 150<br>ngen = 600<br>cxpb = 0.2<br>mutpb = 0.8<br></td><td rowspan="3">intorno ai 2 min <br>3.500.000, <br>converge in <br>circa 10 min </td><td rowspan="3">5.308.034</td></tr><tr><td>100 iter - 5.328.296</td><td>6 min </td></tr><tr><td>200 iter - 5.332.987</td><td>12 min </td></tr><tr><td>TOTAL SCORE</td><td>26.724.326</td><td>//</td><td>20.580.150</td><td>//</td><td>26.884.761</td><td></td></tr></tbody></table>